In [52]:
import pandas as pd

In [53]:
keytable = pd.read_csv("H358_keytable.txt", sep="\t")
featureCount = pd.read_csv("runs_20210209-results_featureCounts_merged_gene_counts.txt", sep="\t")

In [54]:
featureCount = featureCount.reindex(sorted(featureCount.columns), axis=1)
fc_columns = featureCount.columns
sample_names = fc_columns[0:-2]
sample_names

Index(['E2-88342_S80_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88343_S81_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88344_S82_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88345_S83_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88346_S84_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88347_S85_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88348_S86_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88349_S87_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88350_S88_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88351_S89_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88352_S90_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88353_S91_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88354_S92_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88355_S93_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88356_S94_L004_R1_001Aligned.sortedByCoord.out.bam',
       'E2-88357_S95_L004_R1_001Aligned.

In [55]:
#sample_list=[]
#for name in sample_names:
 #   sample_ID = name[0:8]
  #  sample_list.append(sample_ID)

In [56]:
file_dict = {name:name[0:8] for name in sample_names}

In [57]:
keytable["SampleID-Lab"]=keytable["Description"].str[-10:]
keytable.rename(columns={"Sample_ID":"SampleID"}, inplace=True)

In [58]:
lab_ID = keytable.loc[lambda df: df['SampleID'] == sample_list[0], ["SampleID-Lab"]]
print(lab_ID)
ID = lab_ID.iloc[0:1]
ID

  SampleID-Lab
0   ES58NTXX01


,SampleID-Lab
0,ES58NTXX01


In [59]:
keytable["SampleID"][0]

'E2-88342'

In [60]:
ID_dict = {keytable["SampleID"][row]:keytable["SampleID-Lab"][row] for row in keytable.index}

In [61]:
featureCount.rename(columns=file_dict, inplace=True)
featureCount.rename(columns=ID_dict, inplace=True)

In [62]:
#lab_sample_list=[]
#for sample in sample_list:
#    lab_ID = keytable.loc[keytable['SampleID'] == sample, "SampleID-Lab"].values[0]
#    lab_sample_list.append(lab_ID)

In [63]:
#rearrange columns to put geneid and gene name at front
columns = featureCount.columns.tolist()
columns = columns[-2:] + columns[:-2]

In [64]:
featureCount=featureCount[columns]

In [24]:
#write relabeled featureCounts to tsv file
featureCount.to_csv(path_or_buf = "20210209-featureCounts-labIDs.tsv", sep="\t")

In [65]:
exp_design = pd.DataFrame()
exp_design["SampleID-Lab"]=keytable["SampleID-Lab"]

In [66]:
info = ["Cell Line", "Inhibition Status", "CRISPR", "MRTX", "BI", "SHP2i", "Time Point", "Population"]
def extract_info(ID, info):
    if info == "Cell Line":
        return ID[2:4]
    if info == "CRISPR":
        return ID[4:6]
    if len(ID) != 10:
        if info == "Inhibition Status":
            return ID[4:6] + "XX"
        if info == "MRTX" or info == "BI" or info == "SHP2i":
            return False
        if info == "Time Point":
            return 0
        if info == "Population":
            return "A"
    else:
        if info == "Inhibition Status":
            return ID[4:8]
        if info == "MRTX":
            if "M" in ID[6:8]:
                return True
            else:
                return False
        if info == "BI":
            if "B" in ID[6:8]:
                return True
            else:
                return False
        if info == "SHP2i":
            if "S" in ID[6:8]:
                return True
            else:
                return False
        if info == "Time Point":
            if int(ID[8:10]) <= 4 and ID[6:8]=="XX":
                return 0
            elif int(ID[8:10]) <= 4:
                return 6
            else: 
                return 72
        if info == "Population":
            if int(ID[8:10]) in [1,2,5,6]:
                return "A"
            else:
                return "B"


In [67]:
for x in info:
    exp_design[x]=exp_design["SampleID-Lab"].apply(lambda row: extract_info(row, x))

In [29]:
#write experimental design to tsv file
exp_design.to_csv(path_or_buf = "20201009-H358-experiment-design.tsv", sep="\t")

In [30]:
#isolate H23s
#exp_design_H23 = exp_design[exp_design["SampleID-Lab"].str.contains("ES35")==False]

In [31]:
#exp_design_H23.to_csv(path_or_buf = "20201027-H23-experiment-design.tsv", sep="\t")

In [39]:
#get list of RTKs from text file from grant Fig 2
with open("RTKs.txt") as f:
    RTKs = f.read().splitlines()
while("" in RTKs) : 
    RTKs.remove("") 

In [40]:
#get list of RTK ligands from text file from grant Fig 2
with open("RTK-ligands.txt") as f:
    RTK_ligands = f.read().splitlines()
while("" in RTK_ligands) : 
    RTK_ligands.remove("") 

In [41]:
#all_genes = RTKs + RTK_ligands

In [42]:
good_genes = []
#check that genes are named in featureCount
print("These RTKs/RTK ligands are not listed in featureCount:")
for gene in RTKs:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These RTKs/RTK ligands are not listed in featureCount:
PDDGRB
RYIK


In [43]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_RTK = featureCount[featureCount["gene_name"].isin(good_genes)]

In [44]:
#isolate H23s in featureCount_RTK
#featureCount_RTK_H23 = featureCount_RTK.loc[:,featureCount_RTK.columns.str.contains("ES35")==False]

In [50]:
#write featureCount_RTK_H23 to tsv
featureCount_RTK.to_csv(path_or_buf = "20210209-featureCounts-H358-RTK.tsv", sep="\t")

In [46]:
good_genes = []
#check that genes are named in featureCount
print("These RTKs/RTK ligands are not listed in featureCount:")
for gene in RTK_ligands:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These RTKs/RTK ligands are not listed in featureCount:
SCF
CLC11A


In [47]:
#make featureCount dataframe with existing RTK/RTK ligand genes
featureCount_RTK_lig = featureCount[featureCount["gene_name"].isin(good_genes)]

In [48]:
#isolate H23s in featureCount_RTK
#featureCount_RTK_H23 = featureCount_RTK.loc[:,featureCount_RTK.columns.str.contains("ES35")==False]

In [51]:
#write featureCount_RTK_H23 to tsv
featureCount_RTK_lig.to_csv(path_or_buf = "20210209-featureCounts-H358-RTK_lig.tsv", sep="\t")

In [38]:
#featureCount_RTK_H23.loc[featureCount_RTK_H23["Geneid"] == "ENSG00000124882"]

In [35]:
#ERK Signature gene list
with open("ERK_KEGG_geneset.txt") as f:
    ERK_KEGG = f.read().splitlines()


In [36]:
ERK_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in ERK_KEGG:
    if gene in featureCount["gene_name"].tolist():
        ERK_genes.append(gene)
    else:
        print(gene)

These RTKs/RTK ligands are not listed in featureCount:
MAP3K20


In [37]:
#make featureCount dataframe with existing ERK genes
featureCount_ERK = featureCount[featureCount["gene_name"].isin(good_genes)]
#write featureCount_RTK_H23 to tsv
featureCount_ERK.to_csv(path_or_buf = "20210209-featureCounts-H358-ERK.tsv", sep="\t")

In [68]:
#Ephrin gene list
with open("Ephrin genes txt.txt") as f:
    Ephrins = f.read().splitlines()


In [69]:
good_genes = []
#check that genes are named in featureCount
print("These genes are not listed in featureCount:")
for gene in Ephrins:
    if gene in featureCount["gene_name"].tolist():
        good_genes.append(gene)
    else:
        print(gene)

These genes are not listed in featureCount:


In [70]:
#make featureCount dataframe with existing ERK genes
featureCount_ephrin = featureCount[featureCount["gene_name"].isin(good_genes)]
#write featureCount_RTK_H23 to tsv
featureCount_ephrin.to_csv(path_or_buf = "20210209-featureCounts-H358-ephrin.tsv", sep="\t")